In [19]:
import numpy as np 
import pandas as pd 
import tensorflow as tf 
import matplotlib.pyplot as plt 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler

In [20]:
def load_dataset(csv_path):
    df = pd.read_csv(csv_path, sep= ';')
    required_columns = ['Azimuth', 'Altitude', 'Timestamp']

    if not all (col in df.columns for col in required_columns):
        raise ValueError("Required columns are not present in the DataFrame.")
    
    X = df[['Timestamp','Azimuth','Altitude']].values
    y = df[['Azimuth','Altitude']].values

    return X, y, df

In [21]:
csv_path = 'cleaned/cleaned_data_quartal_1_2020.csv'
X, y, df = load_dataset(csv_path)

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2 , random_state= 42)

In [23]:
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_train_scaled = scaler_X.fit_transform(X_train)
y_train_scaled = scaler_y.fit_transform(y_train)
X_test_scaled = scaler_X.transform(X_test)
y_test_scaled = scaler_y.transform(y_test)

In [24]:
timesteps = 1
X_train_reshaped = X_train_scaled.reshape((X_train_scaled.shape[0], timesteps, X_train_scaled.shape[1]))
X_test_reshaped = X_test_scaled.reshape((X_test_scaled.shape[0], timesteps, X_test_scaled.shape[1]))

In [25]:
model = Sequential()
model.add(LSTM(units=1024, activation='relu', input_shape=(timesteps, X_train_scaled.shape[1]), return_sequences=True))
model.add(LSTM(units=512, activation='relu', return_sequences=True))
model.add(LSTM(units=256, activation='relu', return_sequences=True))
model.add(LSTM(units=128, activation='relu', return_sequences=True))
model.add(LSTM(units=64, activation='relu', return_sequences=True))
model.add(LSTM(units=32, activation='relu'))
model.add(Dense(units=2))

optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])  # Using Mean Absolute Error as a metric for regression

history = model.fit(X_train_reshaped, y_train_scaled, epochs=500, batch_size=512, validation_data=(X_test_reshaped, y_test_scaled))

Epoch 1/500
21/21 [==============================] - 10s 107ms/step - loss: 0.3089 - mae: 0.5005 - val_loss: 0.3096 - val_mae: 0.5009
Epoch 2/500
21/21 [==============================] - 1s 57ms/step - loss: 0.3058 - mae: 0.4974 - val_loss: 0.3065 - val_mae: 0.4979
Epoch 3/500
21/21 [==============================] - 1s 55ms/step - loss: 0.3027 - mae: 0.4943 - val_loss: 0.3034 - val_mae: 0.4947
Epoch 4/500
21/21 [==============================] - 1s 57ms/step - loss: 0.2996 - mae: 0.4911 - val_loss: 0.3001 - val_mae: 0.4914
Epoch 5/500
21/21 [==============================] - 1s 53ms/step - loss: 0.2963 - mae: 0.4877 - val_loss: 0.2967 - val_mae: 0.4880
Epoch 6/500
21/21 [==============================] - 1s 57ms/step - loss: 0.2928 - mae: 0.4842 - val_loss: 0.2931 - val_mae: 0.4843
Epoch 7/500
21/21 [==============================] - 1s 53ms/step - loss: 0.2890 - mae: 0.4803 - val_loss: 0.2891 - val_mae: 0.4801
Epoch 8/500
21/21 [==============================] - 1s 54ms/step - loss: 

In [26]:
predicted_values_scaled = model.predict(X_test_reshaped)
predicted_values = scaler_y.inverse_transform(predicted_values_scaled)
mse = mean_squared_error(y_test_scaled, predicted_values_scaled)
r2 = r2_score(y_test_scaled, predicted_values_scaled)

accuracy = model.evaluate(X_test_reshaped, y_test_scaled)

82/82 [==============================] - 1s 9ms/step - loss: 1.9813e-06 - mae: 0.0011


In [27]:
timestamps_test = X_test[:, 0]
df_results = pd.DataFrame({
    'Timestamp': timestamps_test,
    'Actual_Azimuth': y_test[:, 0],
    'Actual_Altitude': y_test[:, 1],
    'Predicted_Azimuth': predicted_values[:, 0],
    'Predicted_Altitude': predicted_values[:, 1]
})


In [28]:
output_csv_path = 'azimuth_altitude_prediction_results.csv'
df_results.to_csv(output_csv_path, index=False)

In [29]:
print(f"Mean Squared Error on Test Data: {mse}")
print(f"R-squared on Test Data: {r2}")
print(f"Accuracy on Test Data: {accuracy}")
print(f"Results saved to: {output_csv_path}")

Mean Squared Error on Test Data: 1.9812970768489125e-06
R-squared on Test Data: 0.999959164835712
Accuracy on Test Data: [1.9812969185295515e-06, 0.0011367321712896228]
Results saved to: azimuth_altitude_prediction_results.csv
